In [104]:
import pandas as pd
import numpy as np
from holidays_co import is_holiday_date

In [105]:
df = pd.read_csv('Reporte_Lesiones_Personales_y_en_Accidente_de_Tr_nsito_Polic_a_Nacional.csv', low_memory=False)

In [106]:
df

,DEPARTAMENTO,MUNICIPIO,CODIGO DANE,ARMAS MEDIOS,FECHA HECHO,GENERO,GRUPO ETARIO,DESCRIPCIÓN CONDUCTA,CANTIDAD
0,ANTIOQUIA,GIRARDOTA,5308000,ARMA BLANCA / CORTOPUNZANTE,1/01/2010,FEMENINO,ADULTOS,LESIONES PERSONALES,2
1,ANTIOQUIA,GIRARDOTA,5308000,ARMA BLANCA / CORTOPUNZANTE,1/01/2010,MASCULINO,ADULTOS,LESIONES PERSONALES,1
2,ANTIOQUIA,MUTATÁ,5480000,ARMA BLANCA / CORTOPUNZANTE,1/01/2010,MASCULINO,ADULTOS,LESIONES PERSONALES,1
3,ANTIOQUIA,NECOCLÍ,5490000,ARMA BLANCA / CORTOPUNZANTE,1/01/2010,FEMENINO,ADULTOS,LESIONES PERSONALES,1
4,ATLÁNTICO,BARRANQUILLA (CT),8001000,ARMA BLANCA / CORTOPUNZANTE,1/01/2010,FEMENINO,ADULTOS,LESIONES PERSONALES,2
...,...,...,...,...,...,...,...,...,...
1047244,CESAR,VALLEDUPAR (CT),20001000,VENENO,3/05/2022,MASCULINO,ADULTOS,LESIONES PERSONALES,1
1047245,HUILA,OPORAPA,41503000,VENENO,16/06/2022,FEMENINO,ADOLESCENTES,LESIONES PERSONALES,1
1047246,TOLIMA,IBAGUÉ (CT),73001000,VENENO,17/04/2022,MASCULINO,ADULTOS,LESIONES PERSONALES,1
1047247,CUNDINAMARCA,COTA,25214000,SIN EMPLEO DE ARMAS,30/03/2022,MASCULINO,ADULTOS,LESIONES PERSONALES,1


In [107]:
df = df.dropna()
df = df.drop('CODIGO DANE', axis=1)
df['FECHA HECHO'] = pd.to_datetime(df['FECHA HECHO'], format='%d/%m/%Y')
# Agregamos una columna para dia y otra para mes
df['DIA'] = df['FECHA HECHO'].dt.day
df['MES'] = df['FECHA HECHO'].dt.month
# Agregamos una columna para saber si el dia es festivo si fue festivo se le asigna 1, si no fue festivo se le asigna 0
df['FESTIVO'] = df['FECHA HECHO'].apply(lambda x: 1 if is_holiday_date(x) else 0)
df = pd.DataFrame(df)

In [108]:
# agrupar por grupo etario
df_etario = df.groupby(['GRUPO ETARIO']).count()
df_etario

,DEPARTAMENTO,MUNICIPIO,ARMAS MEDIOS,FECHA HECHO,GENERO,DESCRIPCIÓN CONDUCTA,CANTIDAD,DIA,MES,FESTIVO
GRUPO ETARIO,,,,,,,,,,
ADOLESCENTES,95763,95763,95763,95763,95763,95763,95763,95763,95763,95763
ADULTOS,853564,853564,853564,853564,853564,853564,853564,853564,853564,853564
MENORES,39947,39947,39947,39947,39947,39947,39947,39947,39947,39947
NO REPORTA,81,81,81,81,81,81,81,81,81,81
NO REPORTADO,56930,56930,56930,56930,56930,56930,56930,56930,56930,56930


In [109]:
min = df['FECHA HECHO'].min()
max = df['FECHA HECHO'].max()

In [110]:
data_range = df[(df["FECHA HECHO"] >= min) & (df["FECHA HECHO"] <= max)]
    # Agrupar los datos por fecha y sumar la cantidad
data_xd = data_range.groupby(by="FECHA HECHO")["CANTIDAD"].sum().reset_index()
data_xd

,FECHA HECHO,CANTIDAD
0,2010-01-01,984
1,2010-01-02,264
2,2010-01-03,288
3,2010-01-04,232
4,2010-01-05,209
...,...,...
4621,2022-08-27,224
4622,2022-08-28,263
4623,2022-08-29,175
4624,2022-08-30,168


In [111]:
# sumar las filas que tengan valores repetidos
df_mod = df.groupby(['DEPARTAMENTO', 'MUNICIPIO', 'ARMAS MEDIOS', 'FECHA HECHO', 'GENERO', 'GRUPO ETARIO', 'DESCRIPCIÓN CONDUCTA', 'CANTIDAD', 'DIA', 'MES', 'FESTIVO']).sum().reset_index()

In [112]:
# mostrar las filas en el departamento de Antioquia, municipio de girardota, armas medios arma blanca / cortopunzante, fecha 2010-01-01
df_mod[(df_mod['DEPARTAMENTO'] == 'ANTIOQUIA') & (df_mod['MUNICIPIO'] == 'GIRARDOTA') & (df_mod['ARMAS MEDIOS'] == 'ARMA BLANCA / CORTOPUNZANTE') & (df_mod['FECHA HECHO'] == '2010-01-01')]


,DEPARTAMENTO,MUNICIPIO,ARMAS MEDIOS,FECHA HECHO,GENERO,GRUPO ETARIO,DESCRIPCIÓN CONDUCTA,CANTIDAD,DIA,MES,FESTIVO
37538,ANTIOQUIA,GIRARDOTA,ARMA BLANCA / CORTOPUNZANTE,2010-01-01,FEMENINO,ADULTOS,LESIONES PERSONALES,2,1,1,1
37539,ANTIOQUIA,GIRARDOTA,ARMA BLANCA / CORTOPUNZANTE,2010-01-01,MASCULINO,ADULTOS,LESIONES PERSONALES,1,1,1,1


In [113]:
# Mostrar las diferencias entre los datos originales y los datos modificados
df_mod.shape, df.shape

((1009899, 11), (1046285, 11))

In [114]:
# Mostrar la cantidad por armas medios
df['ARMAS MEDIOS'].value_counts()

CONTUNDENTES                          368412
ARMA BLANCA / CORTOPUNZANTE           247121
VEHICULO                              156174
MOTO                                  106602
SIN EMPLEO DE ARMAS                    53606
ARMA DE FUEGO                          45582
NO REPORTA                             37297
ACIDO                                  13302
PUNZANTES                               4985
BICICLETA                               2288
PERRO                                   2028
ARTEFACTO EXPLOSIVO/CARGA DINAMITA      1100
NO REPORTADO                             903
ARMA TRAUMATICA                          851
COMBUSTIBLE                              714
GRANADA DE MANO                          703
ESCOPOLAMINA                             686
MINA ANTIPERSONA                         549
AGUA CALIENTE                            464
QUIMICOS                                 298
SUSTANCIAS TOXICAS                       256
CUCHILLA                                 242
GASES     

In [115]:
# Mostrar la cantidad por fecha
df['FECHA HECHO'].value_counts()

2020-01-01    1346
2017-01-01     858
2016-01-01     795
2019-01-01     761
2013-01-01     739
              ... 
2020-03-26      63
2021-01-08      60
2021-12-14      57
2020-12-30      52
2020-12-31      48
Name: FECHA HECHO, Length: 4626, dtype: int64

In [116]:
df_date = df.groupby(['FECHA HECHO']).sum().reset_index()
df_date

,FECHA HECHO,CANTIDAD,DIA,MES,FESTIVO
0,2010-01-01,984,506,506,506
1,2010-01-02,264,404,202,0
2,2010-01-03,288,636,212,0
3,2010-01-04,232,708,177,0
4,2010-01-05,209,705,141,0
...,...,...,...,...,...
4621,2022-08-27,224,3780,1120,0
4622,2022-08-28,263,4564,1304,0
4623,2022-08-29,175,3393,936,0
4624,2022-08-30,168,3540,944,0
